In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
from pandas import Series, DataFrame

#Intro, overview

##Why the complexity of Pandas's groupby and MultiIndex, when SQL is simpler?

Per the intro, the complexity of pandas syntax and MultiIndex use is ok because 'query languages like SQL are rather limited in the kinds of group operations that can be performed'. Pandas supports 'much more complex grouped operations by utilizing any function that accepts a pandas object or NumPy array.'

For example, rather than just specifying the names of columns to define how to group, pandas lets you define your group (the keys) in terms of DataFrame column names, arrays, or functions (i.e., in code). Once you have a set of groups, you can compute things - like count, mean, etc. using pre-defined function, or define your own functions.

##Mechanics - split, apply, combine

All group by work can be thought of as having three parts:
1. Split - Data (in a Series, DataFrame, etc.) is split into groups based on one or more keys. SQL splits data into groups by row - i.e., the groups are made up of one or more rows. In Pandas this is splitting by axis = 0. Pandas can also group by/split columns, with axis = 1.
2. Apply - A function is applied to each group, producing a new value. For example, 'count' can be applied to count the number of rows (or columns, if splitting by axis = 1), sum can be applied to sum values associated with each row in the group, etc.
3. Combine - Finally, the results of the apply step are combined into a result object. The book says taht the 'resulting object will usually depend on what's being done to the data' - I'm not sure what this means; hopefully it'll become clear after working through the chapter.

#Split - defining keys

Ultimately you want a set of one or more grouping keys. You can provide explicitly a list or array of values that is the same length as the axis being grouped (for ex, if you're grouping rows then you provide a list w/ values where the length of the list is equal to the number of rows). Or you can provide other stuff, as shown below, that are each and all shortcuts to define a list of values that's the same length as the axis being grouped.

In [4]:
df = DataFrame({'key1': list('aabba'),
                'key2': ['one','two','one','two','one'],
                'data1': np.random.randn(5),
                'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.095599,1.632777,a,one
1,1.262306,0.444808,a,two
2,-0.201121,0.229857,b,one
3,0.882426,-0.766906,b,two
4,1.059917,-0.795773,a,one


As one example - which one, I'll need to think through or learn - here we first get a Series object for the 'data1' column. Then we group that Series object using values from the 'key1' column. I think this means we're working with two Series instances - one that has the data we care about working with, on which we call the groupby method, and one that provides the values that groupby needs to actually group the original values.

In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

Once we have this SeriesGroupBy object, which I think represents the groups but not the apply part of the steps above - i.e., it's just doing the split part, then we can actually do the 'apply' by applying a function to the groupby.

In [6]:
grouped.mean()

key1
a    0.805941
b    0.340653
Name: data1, dtype: float64

When we apply a function like mean we turn a Series into another Series, where the produced/second Series has a row for each unique value specified by the key. Here the key is only made up of the key1 column, and the key1 column has two unique values, so the resulting Series has two rows - further, the index values for these two rows are the unique values. The actual data associated with each row is the result of the function we applied to the SeriesGroupBy object, which is 'mean' in this case.

Another example involves passing multiple arrays as a list.

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.577758
      two     1.262306
b     one    -0.201121
      two     0.882426
Name: data1, dtype: float64

The above is the same as the first example, except that we group by two Series. We get a MultiIndex where the first level is the unique values of the first key, and the second level is the unique values of the second key. 

In [9]:
means.unstack()

key2,one,two
key1,,
a,0.577758,1.262306
b,-0.201121,0.882426


The next example was a bit odd to me, because it shows the use of arrays w/ any content, as long as they're the 'right' length, where 'right' means the same length as the axis we're grouping. Based on checking the values of the actual calculated means, it looks like the _location_ of the key values is what matters... the actual key values themselves aren't important to figure out which rows in the source are grouped... they're only used as the labels for the index in the resulting groupby and DataFrame. That is, in the following example, 'Ohio' and 2005 are at location 0, so the row in the source data at location 0 is made a part of the group with keys 'Ohio' and 2005. Location three also has the same key values, so it's made part of the same group. The other locations have different key values.

In [11]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

df['data1'].groupby([states, years]).mean()

California  2005    1.262306
            2006   -0.201121
Ohio        2005    0.489013
            2006    1.059917
Name: data1, dtype: float64

In all of the above examples we used individual Series instances on which to call groupby and to provide the key(s). There was no relationship needed between the two. 

Another option, if the grouping information is in the same DataFrame as the data being grouped, all you need to do is specify the name of the column(s).

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.805941,0.427271
b,0.340653,-0.268524


Note that in the previous example, there's no 'key2' column in the results, even though we're not grouping by it and it's in the source data frame. This is because by default any columns that have non-numeric data are considered 'nuisance' columns (at least when you do the apply w/ a function - like mean - that requires numeric data?) and so left out of the result.

In [13]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.577758  0.418502
     two   1.262306  0.444808
b    one  -0.201121  0.229857
     two   0.882426 -0.766906

In [14]:
# don't forget that to just get a count of rows/items in the group
# use 'size' (NOT 'count')
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

#Iterating over groups

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  0.095599  1.632777    a  one
1  1.262306  0.444808    a  two
4  1.059917 -0.795773    a  one
b
      data1     data2 key1 key2
2 -0.201121  0.229857    b  one
3  0.882426 -0.766906    b  two


In [17]:
for (k1, k2), group in df.groupby(['key1','key2']):
    print(k1, k2)
    print(group)

a one
      data1     data2 key1 key2
0  0.095599  1.632777    a  one
4  1.059917 -0.795773    a  one
a two
      data1     data2 key1 key2
1  1.262306  0.444808    a  two
b one
      data1     data2 key1 key2
2 -0.201121  0.229857    b  one
b two
      data1     data2 key1 key2
3  0.882426 -0.766906    b  two


A useful recipe is 'computing a dict of the data pieces [i.e., the 'group' part of the tuples returned by groupby) as a one-liner':

In [18]:
pieces = dict(list(df.groupby('key1')))
len(pieces)

2

In [19]:
pieces

{'a':       data1     data2 key1 key2
 0  0.095599  1.632777    a  one
 1  1.262306  0.444808    a  two
 4  1.059917 -0.795773    a  one, 'b':       data1     data2 key1 key2
 2 -0.201121  0.229857    b  one
 3  0.882426 -0.766906    b  two}

In [20]:
pieces['b']

,data1,data2,key1,key2
2,-0.201121,0.229857,b,one
3,0.882426,-0.766906,b,two


#Example of grouping columns instead of rows

In [21]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [22]:
grouped_cols = df.groupby(df.dtypes, axis=1)
grouped_cols

In [23]:
dict(list(grouped))

{'a': 0    0.095599
 1    1.262306
 4    1.059917
 Name: data1, dtype: float64, 'b': 2   -0.201121
 3    0.882426
 Name: data1, dtype: float64}

#Selecting a column or subset of columns

With no further code, the apply method is used on each (non-numeric) column. We can also specify only selected columns.